In [1]:
import pickle

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [2]:
from sklearn.pipeline import make_pipeline

In [3]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5001")
mlflow.set_experiment("green-taxi-duration")

2024/06/16 17:23:54 INFO mlflow.tracking.fluent: Experiment with name 'green-taxi-duration' does not exist. Creating a new experiment.


<Experiment: artifact_location='/home/tsimpson_unix/ml_ops_zoomcamp/w4_deployment/web-service/web-service-mlflow/artifacts/1', creation_time=1718573034100, experiment_id='1', last_update_time=1718573034100, lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [4]:
def read_dataframe(filename: str):
    # Ensure the datetime columns are parsed correctly
    df = pd.read_csv(filename, parse_dates=['lpep_pickup_datetime', 'lpep_dropoff_datetime'])

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df['duration'] = df['duration'].dt.total_seconds() / 60
    df = df[(df['duration'] >= 1) & (df['duration'] <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df

def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts


In [5]:
df_train = read_dataframe('data/green_tripdata_2021-01.csv')
df_val = read_dataframe('data/green_tripdata_2021-02.csv')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

/tmp/ipykernel_96501/2992171150.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename, parse_dates=['lpep_pickup_datetime', 'lpep_dropoff_datetime'])


In [6]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model")

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.756595470642227


/home/tsimpson_unix/anaconda3/envs/ml-ops-zoomcamp-01/lib/python3.9/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [7]:
from mlflow.tracking import MlflowClient


In [10]:
MLFLOW_TRACKING_URI = 'http://127.0.0.1:5001'
RUN_ID = '2f709a96549e4301b9359c10669eca87'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [11]:
path = client.download_artifacts(run_id=RUN_ID, path='dict_vectorizer.bin')

/tmp/ipykernel_96501/2836594815.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.download_artifacts`` is deprecated since 2.0. This method will be removed in a future release. Use ``mlflow.artifacts.download_artifacts`` instead.
  path = client.download_artifacts(run_id=RUN_ID, path='dict_vectorizer.bin')


OSError: No such file or directory: '/home/tsimpson_unix/ml_ops_zoomcamp/w4_deployment/web-service/web-service-mlflow/artifacts/1/2f709a96549e4301b9359c10669eca87/artifacts/dict_vectorizer.bin'

In [28]:
with open(path, 'rb') as f_out:
    dv = pickle.load(f_out)

/home/tsimpson_unix/anaconda3/envs/ml-ops-zoomcamp-01/lib/python3.9/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [29]:
dv

DictVectorizer()